# Smart Substitute Suggestion

**Purpose**  
This notebook demonstrates a pipeline for:
1. **Loading** grocery catalog data from multiple sources (IGA & Woolworths).  
2. **Normalizing** column names and data types for consistency.  
3. **Scoring** items by combining factors like necessity, purchase history, loyalty, affordability, availability, and ratings.  
4. **Finding** TF–IDF–based, cheaper “smart substitutes” in the catalog.  
5. **Simulating** shopping carts and suggesting top replacements.  
6. **Bulk-exporting** simulation results to CSV for downstream analysis.

**Why?**  
- Retailers can use this to automatically suggest lower-cost alternatives to shoppers.  
- Data scientists and engineers can extend or adapt each step for their own catalogs, scoring criteria, or similarity logic.

---

**Notebook structure**  
1. Imports & Setup  
2. Data Loading & Preprocessing  
3. Preference Scoring  
4. Text Cleaning & TF–IDF Similarity  
5. Substitute Suggestion Logic  
6. Cart Simulation & Output  
7. Bulk Simulation & CSV Export  
8. Next Steps & Extensions

## 1. Imports & Setup

We import standard libraries, plus:
- **`tqdm`** for progress bars in loops  
- **`sklearn`** for TF–IDF vectorization and cosine similarity

In [15]:
import os
import re
import random

import numpy as np
import pandas as pd

from collections import Counter
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Ensure reproducibility
random.seed(42)
np.random.seed(42)

## 2. Data Loading & Preprocessing

We support two dataset formats:
- **IGA**: lowercase column names, minimal type coercion  
- **Woolworths**: Title-case column names, explicit type conversions

### 2.1. `load_and_normalize_data()`
- Reads a CSV  
- Infers dataset type from filename  
- Calls `preprocess_data()`

In [16]:
def load_and_normalize_data(filepath):
    """
    Load CSV and standardize to a common schema.

    Args:
        filepath (str): Path to the CSV file.

    Returns:
        df (pd.DataFrame): DataFrame with standardized columns.
        dataset_type (str): 'woolworths' or 'iga'.
    """
    df = pd.read_csv(filepath)
    fname = os.path.basename(filepath).lower()
    dataset_type = (
        'woolworths' if ('woolworths' in fname or 'woolies' in fname) else 'iga'
    )
    df = preprocess_data(df, dataset_type)
    return df, dataset_type

def preprocess_data(df, dataset_type):
    """
    Rename & coerce columns for IGA vs Woolworths.

    Args:
        df (pd.DataFrame): Raw data.
        dataset_type (str): 'iga' or 'woolworths'.

    Returns:
        pd.DataFrame: With columns:
          - 'item_id', 'item_name', 'brand', 'category',
            'unit_price', 'total_price', 'quantity', 'store_name'
    """
    if dataset_type == 'iga':
        df = df.rename(columns={
            'item_id': 'item_id',
            'item_name': 'item_name',
            'brand': 'brand',
            'category': 'category',
            'unit_price': 'unit_price',
            'total_price': 'total_price',
        })
    else:  # woolworths
        df = df.rename(columns={
            'Item ID': 'item_id',
            'Item Name': 'item_name',
            'Brand': 'brand',
            'Category': 'category',
            'Unit Price': 'unit_price',
            'Total': 'total_price',
            'Quantity': 'quantity',
            'Store Name': 'store_name',
        })
        df['item_id'] = df['item_id'].astype(str)
        df['unit_price'] = pd.to_numeric(df['unit_price'], errors='coerce')
        df['total_price'] = pd.to_numeric(df['total_price'], errors='coerce')
    for col in ['quantity', 'store_name']:
        if col not in df.columns:
            df[col] = np.nan
    return df

### 2.2. Quick sanity check

Load a few rows and inspect dtypes to confirm normalization.

In [17]:
# Replace with your local path
iga = "/Users/macbook1/Documents/Submissions/Tri1_25/SIT764/Datamate/IGA/Original_updated_iga_data.csv"
woolies = "/Users/macbook1/Documents/Submissions/Tri1_25/SIT764/Datamate/Test_Data/synthetic_woolworths_cleaned.csv"
file_path = woolies
df_sample, ds_type = load_and_normalize_data(file_path)

print(f"Dataset type: {ds_type}")
print(df_sample.dtypes)
df_sample.head(3)

Dataset type: woolworths
Transaction ID      object
Purchase Date       object
store_name          object
Location            object
Customer ID         object
Gender              object
item_id             object
brand               object
item_name           object
category            object
Product Code         int64
Best Price         float64
unit_price         float64
Discount Amount    float64
quantity             int64
total_price        float64
Payment Type        object
dtype: object


,Transaction ID,Purchase Date,store_name,Location,Customer ID,Gender,item_id,brand,item_name,category,Product Code,Best Price,unit_price,Discount Amount,quantity,total_price,Payment Type
0,WOOL-C57727A12D,2025-03-02,Woolworths,Sydney,WOOL-2AA656C519,Male,A594D12F67,Cavendish,Cavendish Bananas Each,Fruit & Veg,133211,0.81,0.81,0.0,6,4.86,Credit card
1,WOOL-3DAA220567,2024-11-10,Woolworths,Sydney,WOOL-E96BDA36D1,Other,C87ED7067A,Fresh,Fresh Tomato Each,Fruit & Veg,134034,0.65,0.65,0.0,2,1.30,Store card
2,WOOL-6E24CDF252,2024-12-23,Metro,Canberra,WOOL-00BC858F70,Other,0E51767189,Fresh,Fresh Pink Lady Apples Each,Fruit & Veg,105919,0.86,0.86,0.0,1,0.86,Store card


## 3. Preference Scoring

We assign each item a **`preference_score`** combining six sub-scores:

| Sub-score              | Description                                                    | Weight |
|------------------------|----------------------------------------------------------------|--------|
| **Necessity**          | 1 if category ∈ essential_categories, else 0                   | 2.0    |
| **Purchase freq.**     | `purchase_count / max_count` (0–1)                             | 2.0    |
| **Brand loyalty**      | Fraction of past purchases belonging to this item’s brand      | 1.5    |
| **Common brand**       | 1 if brand ∈ top-3 frequent brands, else 0                     | —      |
| **Affordability**      | Inverse price percentile within category                       | 1.0    |
| **Availability**       | Logistic(stock_level) to reward in-stock items                 | 0.5    |
| **Rating**             | `avg_rating × log(1 + review_count)/log(1 + max_reviews)`      | 1.0    |

### 3.1. `compute_brand_scores()`
- Builds `brand_counter` from purchase history  
- Annotates `df['purchase_count']`

### 3.2. `calculate_scores()`
- Computes each sub-score  
- Aggregates with weights → `preference_score`

In [18]:
def compute_brand_scores(df, user_purchase_history):
    """
    Build brand_counter & annotate df['purchase_count'].

    Args:
      df: DataFrame with 'item_id'
      user_purchase_history: dict[item_id->count] or dict[item_id->{'count','brand'}]

    Returns:
      brand_counter (Counter), past_brands (set)
    """
    if not user_purchase_history:
        df['purchase_count'] = 0
        return Counter(), set()
    sample = next(iter(user_purchase_history.values()))
    is_detailed = isinstance(sample, dict)
    if is_detailed:
        df['purchase_count'] = df['item_id'].map(
            lambda pid: user_purchase_history.get(pid, {}).get('count', 0)
        )
        brand_counter = Counter(
            e['brand'] for e in user_purchase_history.values() if 'brand' in e
        )
    else:
        df['purchase_count'] = df['item_id'].map(user_purchase_history)
        brand_counter = Counter()
    return brand_counter, set(brand_counter)

def calculate_scores(df, essential_categories, brand_counter,
                     past_brands, stock_info, rating_info):
    """
    Compute and return df with added score columns.

    Args:
      essential_categories (set): e.g. {'Dairy','Bakery'}
      stock_info (dict): item_id -> int stock level
      rating_info (dict): item_id -> (avg_rating, num_reviews)

    Returns:
      scored_df (pd.DataFrame)
    """
    # sub-scores
    df['necessity_score'] = df['category'].isin(essential_categories).astype(float)
    max_freq = max(df['purchase_count'].max(), 1)
    df['purchase_frequency_score'] = df['purchase_count'] / max_freq
    total_brand = sum(brand_counter.values()) or 1
    df['brand_loyalty_score'] = df['brand'].map(
        lambda b: brand_counter[b] / total_brand if b in brand_counter else 0
    )
    top_brands = {b for b, _ in brand_counter.most_common(3)}
    df['common_brand_score'] = df['brand'].isin(top_brands).astype(float)
    df['affordability_score'] = (
        1 - df.groupby('category')['unit_price'].rank(pct=True)
    ).clip(0, 1)
    if stock_info:
        df['availability_score'] = df['item_id'].map(
            lambda pid: 1/(1 + np.exp(-stock_info.get(pid, 1)))
        )
    else:
        df['availability_score'] = 1.0
    if rating_info:
        max_rev = max(v[1] for v in rating_info.values()) or 1
        df['rating_score'] = df['item_id'].map(
            lambda pid: (
                rating_info[pid][0] * 
                (np.log(1+rating_info[pid][1]) / np.log(1+max_rev))
            ) if pid in rating_info else 0.5
        )
    else:
        df['rating_score'] = 0.5
    df['preference_score'] = (
        2.0 * df['necessity_score'] +
        2.0 * df['purchase_frequency_score'] +
        1.5 * df['brand_loyalty_score'] +
        1.0 * df['affordability_score'] +
        0.5 * df['availability_score'] +
        1.0 * df['rating_score']
    )
    return df

def add_smart_preference_scores_fast(
    df, user_purchase_history=None,
    essential_categories=None, stock_info=None,
    rating_info=None
):
    """Wrapper: computes brand scores & full preference_score."""
    df2 = df.copy()
    bc, pb = compute_brand_scores(df2, user_purchase_history or {})
    return calculate_scores(
        df2, essential_categories or set(),
        bc, pb, stock_info, rating_info
    )

### 3.3. Example: assign scores

In [19]:
history = {'1001': 3, '1002': 1}
stock   = {'1001': 5, '1003': 0}
ratings = {'1001': (4.5, 10), '1002': (3.0, 2)}

df_scored = add_smart_preference_scores_fast(
    df_sample.head(10),
    user_purchase_history=history,
    essential_categories={'Dairy', 'Bakery'},
    stock_info=stock,
    rating_info=ratings
)
df_scored[[
    'item_id', 'item_name', 'necessity_score',
    'purchase_frequency_score', 'affordability_score',
    'availability_score', 'rating_score', 'preference_score'
]]

,item_id,item_name,necessity_score,purchase_frequency_score,affordability_score,availability_score,rating_score,preference_score
0,A594D12F67,Cavendish Bananas Each,0.0,NaN,0.5,0.731059,0.5,NaN
1,C87ED7067A,Fresh Tomato Each,0.0,NaN,0.7,0.731059,0.5,NaN
2,0E51767189,Fresh Pink Lady Apples Each,0.0,NaN,0.3,0.731059,0.5,NaN
3,4FF52DABB3,Strawberries 250g Punnet,0.0,NaN,0.0,0.731059,0.5,NaN
4,7F0EA7ACEE,Hass Avocado Each,0.0,NaN,0.1,0.731059,0.5,NaN
5,A96AEAFD54,Potato White Washed Each,0.0,NaN,0.5,0.731059,0.5,NaN
6,9E796A5CD4,Mandarin Amorette Seedless Each,0.0,NaN,0.9,0.731059,0.5,NaN
7,CE64D76063,Onion Brown Each,0.0,NaN,0.8,0.731059,0.5,NaN
8,47F99CDD51,Fresh Broccoli Each,0.0,NaN,0.2,0.731059,0.5,NaN
9,19D736DCF6,Eat Later Cavendish Bananas Each,0.0,NaN,0.5,0.731059,0.5,NaN


## 4. Text Cleaning & TF–IDF Similarity

To find textual matches:
1. **Clean** both query & catalog names (lowercase, remove brand, strip punctuation).  
2. **Vectorize** with TF–IDF to capture word importance.  
3. **Compute** cosine similarity.

In [20]:
def clean_name(name, brand):
    s = name.casefold()
    b = brand.casefold() if isinstance(brand, str) else ''
    if b:
        s = re.sub(rf'\b{re.escape(b)}\b', '', s)
    return re.sub(r'[^a-z0-9\s]', '', s).strip()

def is_same_product(r, q):
    return r['item_name'].strip().lower() == q['item_name'].strip().lower()

## 5. Substitute Suggestion Logic

`suggest_cheaper_alternatives()` performs a **two-pass** search:
- **Strict**: high similarity & meaningful absolute saving  
- **Fallback**: looser similarity & modest relative saving  

Results are **ranked** by:
1. Priority (quantity & unit matches)  
2. Higher preference_score  
3. Higher text similarity

In [21]:
def get_priority(item, alt):
    same_qty  = alt.get('quantity') == item.get('quantity')
    same_unit = alt.get('unit_type') == item.get('unit_type')
    same_sub  = alt.get('sub_category') == item.get('sub_category')
    same_cat  = alt.get('category') == item.get('category')
    if same_qty and same_unit:
        return 1 if same_sub else (2 if same_cat else 5)
    if same_sub: return 3
    if same_cat: return 4
    return 5

def suggest_cheaper_alternatives(
    item_row, df, price_diff_threshold=0.5,
    similarity_threshold=0.2, max_suggestions=3,
    fallback_similarity=0.0, fallback_price_buffer=0.15
):
    clean_q = clean_name(item_row['item_name'], item_row['brand'])
    df2 = df.copy()
    df2['cleaned_name'] = df2.apply(
        lambda r: clean_name(r['item_name'], r['brand']), axis=1
    )
    corpus = df2['cleaned_name'].tolist() + [clean_q]
    tfidf = TfidfVectorizer().fit_transform(corpus)
    sims = cosine_similarity(tfidf[-1], tfidf[:-1])[0]

    def collect(min_sim, max_price):
        out = []
        seen = set()  # Track seen items
        for i, sim in enumerate(sims):
            row = df2.iloc[i]
            key = (row['item_name'], row.get('store_name', ''))
            if sim < min_sim or is_same_product(row, item_row) or row['unit_price'] > max_price:
                continue
            if key in seen:
                continue
            seen.add(key)
            out.append({
                'cheaper': row['item_name'],
                'unit_price': row['unit_price'],
                'store': row.get('store_name', ''),
                'priority': get_priority(item_row, row),
                'pref_score': round(row['preference_score'], 3),
                'sim': round(sim, 3),
            })
        return out

    strict_max = item_row['unit_price'] - price_diff_threshold
    strict = collect(similarity_threshold, strict_max)
    if strict:
        return sorted(strict, key=lambda x: (x['priority'], -x['pref_score'], -x['sim']))[:max_suggestions]
    
    fallback_max = item_row['unit_price'] * (1 + fallback_price_buffer)
    fallback = collect(fallback_similarity, fallback_max)
    if fallback:
        return sorted(fallback, key=lambda x: (x['priority'], -x['pref_score'], -x['sim']))[:max_suggestions]
    
    return f"No alternatives found for {item_row['item_name']}"

## 6. Cart Simulation & Reporting

- **`generate_random_cart()`** picks random items.  
- **`test_cart_with_alternatives()`** shows the cart & fetches suggestions.  
- **`pretty_print_results()`** formats output nicely.

In [22]:
def generate_random_cart(df, cart_size=5):
    return df.sample(cart_size).to_dict('records')

def test_cart_with_alternatives(df, cart_size=5):
    cart = generate_random_cart(df, cart_size)
    print("\n🛒 Test Cart:")
    for it in cart:
        print(f" - {it['item_name']} @ ${it['unit_price']:.2f} x {it.get('quantity','?')}")
    results = {it['item_name']: {'orig': it, 'alts': suggest_cheaper_alternatives(it, df)} for it in cart}
    return results

def pretty_print_results(results):
    for name, data in results.items():
        o = data['orig']
        print(f"\n{name} — ${o['unit_price']:.2f}")
        alts = data['alts']
        if isinstance(alts, str):
            print("  ", alts)
        else:
            for i, a in enumerate(alts,1):
                print(f"  {i}. {a['cheaper']} @ ${a['unit_price']:.2f} (Priority {a['priority']}, sim {a['sim']}, pref {a['pref_score']})")

### 6.1. Demo run

In [23]:
# compute scores on full sample
df_full = add_smart_preference_scores_fast(df_sample)
res_demo = test_cart_with_alternatives(df_full, cart_size=4)
pretty_print_results(res_demo)


🛒 Test Cart:
 - Dare Espresso Ice Coffee 750ml @ $5.35 x 5
 - Oat Milk Goodness Omg Chocolate Protein Oat Milk 350ml @ $4.50 x 2
 - Sweet Baby Ray's Sweet N Spicy Bbq Sauce 425ml @ $7.50 x 6
 - Likofu Custard Buns 8 Pack 240g @ $5.30 x 5

Dare Espresso Ice Coffee 750ml — $5.35
  1. Woolworths Peppermint Chocolate Flakes Ice Cream Sandwich 4 Pack @ $3.50 (Priority 1, sim 0.201, pref 1.97)
  2. La Ice Cola Bottle 2l @ $2.00 (Priority 1, sim 0.272, pref 1.964)
  3. La Ice Cola No Sugar 2l @ $2.00 (Priority 1, sim 0.244, pref 1.964)

Oat Milk Goodness Omg Chocolate Protein Oat Milk 350ml — $4.50
  1. Woolworths Skim Milk 1l @ $1.60 (Priority 1, sim 0.212, pref 1.986)
  2. Pandaroo Coconut Milk 400ml @ $1.65 (Priority 1, sim 0.231, pref 1.972)
  3. Devondale Skim Milk Long Life Uht Milk 1l @ $2.20 (Priority 1, sim 0.251, pref 1.959)

Sweet Baby Ray's Sweet N Spicy Bbq Sauce 425ml — $7.50
  1. Mccain Peas Baby 500g @ $3.90 (Priority 1, sim 0.237, pref 1.949)
  2. Woolworths Whole Baby Carro

## 7. Bulk Simulation & CSV Export

Generate *N* carts of size 3–5, take only the **top** suggestion per item, and export to CSV.

In [24]:
if __name__ == "__main__":
    df_run = add_smart_preference_scores_fast(df_sample)
    rows = []
    for cart_id in tqdm(range(1, 101)):
        size = random.randint(3, 5)
        cart_res = test_cart_with_alternatives(df_run, size)
        for name, data in cart_res.items():
            orig = data['orig']
            alts = data['alts']
            top = alts[0] if isinstance(alts, list) and alts else {}
            rows.append({
                'cart_id': cart_id,
                'original_item': name,
                'orig_price': orig['unit_price'],
                'alt_item': top.get('cheaper', 'N/A'),
                'alt_price': top.get('unit_price'),
                'priority': top.get('priority'),
            })
    out_df = pd.DataFrame(rows)
    out_path = f"Results/smart_substitutes_{ds_type}.csv"
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    out_df.to_csv(out_path, index=False)
    print(f"Exported to {out_path}")

  0%|          | 0/100 [00:00<?, ?it/s]


🛒 Test Cart:
 - Little Moons Mochi Bites Strawberries & Cream 6 Pack @ $10.00 x 5
 - Praise Seafood Sauce 250ml @ $3.50 x 4
 - Fibre One 90 Calorie Lemon Drizzle Snack Bars 5 Pack @ $6.90 x 2
 - Bertocchi Gold Short Rindless Bacon 400g @ $11.00 x 4
 - Masterfoods Dried Italian Herbs 10g @ $2.70 x 2


  1%|          | 1/100 [00:19<32:53, 19.94s/it]


🛒 Test Cart:
 - Maggie Beer Quince Paste 100g @ $6.30 x 3
 - Kenco Jumbo Sponge Each @ $4.00 x 7
 - Red Rock Deli Classics Sour Cream & Chives 190g @ $9.60 x 8


  2%|▏         | 2/100 [00:31<24:19, 14.89s/it]

...


🛒 Test Cart:
 - Passage To Asia Satay Chicken 200g @ $3.80 x 7
 - Delta Chicco D'oro Ground Coffee 1kg @ $15.00 x 10
 - Mission Salt Reduced Tortillas 12 Pack @ $4.50 x 7
 - Tia Maria Liqueur 700ml @ $46.00 x 3


 99%|█████████▉| 99/100 [26:45<00:16, 16.05s/it]


🛒 Test Cart:
 - Dettol Antibacterial Liquid Hand Wash Hand Soap Refill Refresh 950ml @ $10.00 x 4
 - Macro Organic Fairtrade Coffee Beans Medium 200g @ $6.60 x 2
 - Woolworths Popcorn Lightly Salted 80g @ $2.10 x 4
 - Mclintocks Vanilla Fresh Fridge Wipe & Deodorizer 500ml @ $5.25 x 3


100%|██████████| 100/100 [27:01<00:00, 16.22s/it]

Exported to Results/smart_substitutes_woolworths.csv


## 8. Next Steps & Extensions

- **Custom scoring**: add sustainability, nutritional info, or promo flags  
- **Advanced NLP**: use word embeddings or sentence transformers for similarity  
- **UI integration**: wrap into a web service or in-app recommendation widget  
- **A/B testing**: validate substitute suggestions with real user engagement data  

*Feel free to adapt each cell to your data schema and business rules!*